These data come from a survey conducted by INSEE, Budget des Familles, which are protected by the Statistical Secrecy Committee. To reproduce the analysis, one must request access to this data.

In [44]:
import pandas as pd

In [45]:
df = pd.read_csv('BDF/Csv/C05.csv', sep=";", index_col="IDENT_MEN")

We need to remove the columns starting with 13 (taxes) and 14 (subsidies) which are not consumption. And also the columns total CTOT and pondmen (weight household) which are not useful.

In [46]:
columns_to_delete = df.columns[df.columns.str.startswith('C13') | df.columns.str.startswith('C14')]
df = df.drop(columns=columns_to_delete)
df = df.drop(columns=["CTOT", "pondmen"])

In [47]:
df_prop = df.divide(df.sum(axis=1), axis=0)


In [48]:
df_prop.rename(columns=lambda x: x.replace('C', ''), inplace=True)


We know have the proportion of each consumption area per household.

In [49]:
import eurostat

In [50]:
df_inflation_raw = eurostat.get_data_df(code="PRC_HICP_MANR", filter_pars={
                                    'geo': 'FR', 'startPeriod': "2021-03"})
df_inflation_raw = df_inflation_raw.rename(columns={'geo\TIME_PERIOD': 'geo'})

In [75]:
df_correspondances = pd.read_csv("correspondances.csv", dtype="str", sep=";")

In [76]:
df_correspondances.set_index('colonnes de df_prop', inplace=True)

In [54]:
df_inflation = df_inflation_raw[df_inflation_raw['coicop'].str.startswith('CP')]

In [55]:
df_inflation['coicop'] = df_inflation['coicop'].str.replace('CP', '')


C:\Users\mdaversa\AppData\Local\Temp\ipykernel_16448\2914206753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inflation['coicop'] = df_inflation['coicop'].str.replace('CP', '')


In [56]:
df_inflation = df_inflation.loc[:, ["coicop", "2023-04"]]


In [57]:
df_inflation = df_inflation.set_index('coicop')

In [77]:
def cell_func(cell_value, col_name):
    # Votre logique ici, vous pouvez utiliser 'col_name' et 'cell_value'
    coicop = df_correspondances.loc[col_name]['colonnes de df_inflation correspondante']
    return df_inflation.loc[coicop]["2023-04"]

def col_func(col):
    column_name = col.name
    return col.apply(cell_func, args=(column_name,))


test = df_prop.apply(col_func)

In [ ]:
missing_columns = set(df_prop.columns) - set(df_inflation.columns)
print(sorted(missing_columns))


In [ ]:
missing_columns2 = set(df_inflation.columns) - set(df_prop.columns)
print(sorted(missing_columns2))

In [ ]:
df_prop_inflation = df_prop*df_inflation